In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import AutoTokenizer

model_name = "google-bert/bert-base-uncased"

In [3]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_BoolQ")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def preprocess_data(examples):
    passage_inputs = [f"passage : {x} " for x in examples["passage"]]
    question_inputs = [f"question : {x} " for x in examples["question"]]
    inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

    model_inputs = tokenizer(inputs, max_length=400, truncation=True)

    model_inputs["labels"] = examples["label"]
    model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

    return model_inputs

In [6]:
tokenized_dataset = dataset.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [9]:
from transformers import AutoModelForSequenceClassification, set_seed

set_seed(42)

id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments

new_model_name = "fine_tuned_boolq_bert"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 0.6443, 'grad_norm': 20.625864028930664, 'learning_rate': 1.7500000000000002e-05, 'epoch': 4.17}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.560615599155426, 'eval_accuracy': 0.7777777777777778, 'eval_f1': 0.6805555555555556, 'eval_runtime': 0.5099, 'eval_samples_per_second': 35.303, 'eval_steps_per_second': 5.884, 'epoch': 4.17}
{'loss': 0.3932, 'grad_norm': 18.010740280151367, 'learning_rate': 1.5000000000000002e-05, 'epoch': 8.33}


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.5756038427352905, 'eval_accuracy': 0.6666666666666666, 'eval_f1': 0.6666666666666666, 'eval_runtime': 0.7047, 'eval_samples_per_second': 25.543, 'eval_steps_per_second': 4.257, 'epoch': 29.17}
{'loss': 0.0016, 'grad_norm': 0.043093618005514145, 'learning_rate': 0.0, 'epoch': 33.33}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.5736490488052368, 'eval_accuracy': 0.7222222222222222, 'eval_f1': 0.7325102880658435, 'eval_runtime': 0.7349, 'eval_samples_per_second': 24.494, 'eval_steps_per_second': 4.082, 'epoch': 33.33}
{'train_runtime': 256.4054, 'train_samples_per_second': 12.48, 'train_steps_per_second': 1.56, 'train_loss': 0.1487432058341801, 'epoch': 33.33}


TrainOutput(global_step=400, training_loss=0.1487432058341801, metrics={'train_runtime': 256.4054, 'train_samples_per_second': 12.48, 'train_steps_per_second': 1.56, 'total_flos': 564389658188160.0, 'train_loss': 0.1487432058341801, 'epoch': 33.333333333333336})

In [13]:
trainer.evaluate()

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.5736490488052368,
 'eval_accuracy': 0.7222222222222222,
 'eval_f1': 0.7325102880658435,
 'eval_runtime': 0.7476,
 'eval_samples_per_second': 24.077,
 'eval_steps_per_second': 4.013,
 'epoch': 33.333333333333336}

In [14]:
trainer.push_to_hub(new_model_name)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/fine_tuned_boolq_bert/commit/530eae0dbb24577bc95984c5517ac4ca38446222', commit_message='fine_tuned_boolq_bert', commit_description='', oid='530eae0dbb24577bc95984c5517ac4ca38446222', pr_url=None, pr_revision=None, pr_num=None)

In [15]:
from transformers import pipeline

adapter_name = "lenatr99/" + new_model_name

# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

[{'label': 'True', 'score': 0.9652435183525085}]